# Group 11 DL Project - Part A 

# Imports 

In [1]:
import gensim
import numpy as np
import pandas as pd
import sklearn as sk
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, ConcatDataset
from os import listdir
from os.path import isfile, join
import re
import nltk
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))
####
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import torch.optim as optim
import time
from sklearn.metrics import roc_curve, auc


## Data Preprocessing

In [2]:
# Now we will make a new dataset from the files with positive and negative reviews

def make_clean_comments(dir_path, comment_type):
    #get all the files from the directory
    onlyfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]
    files_paths = [dir_path + "\\" + f for f in onlyfiles]
    
    comments_tokens = [] #{[tokens_from_1_file]:0/1,[tokens_from_2_file]...} e.t.c
    regex = re.compile('[^a-zA-Z]') #regex for removing all the symbols that are not letters
    
    #for each comment(=file in pos/neg directory)
    for path_to_file in files_paths:
        with open(path_to_file, encoding="utf8") as f:
            full_comment = f.read() #the data from file with comments
            words = full_comment.split() 
            clean_list = [] #this list will contain only words that doesn't have symbols other than letters
            
            #for each words in comment (=file)
            for w in words:
                clean_word_data = regex.sub('', w) #clean the word by regex
                
                if len(clean_word_data) > 1:
                    #cleaning the stopwords from the sentence
                    clean_word_data = clean_word_data.lower()
                    #checking that the word is not a stop word (=a word that shows up frequently)
                    if clean_word_data not in en_stops:
                        clean_list.append(clean_word_data) #creates a clean comment
                        
        comments_tokens.append(clean_list)
        #creating list of comment type for the later creation of dataset
    if comment_type == "pos":          
        comments_type = [1 for i in range(len(comments_tokens))]
    else:
        comments_type = [0 for i in range(len(comments_tokens))]

    return comments_tokens, comments_type

## Positive comments tokenization ##
pos_dir_path = fr'C:\Users\YuvalZiv\Desktop\nlp project\train\pos'
pos_tokens_sentences, comments_type_list_pos = make_clean_comments(pos_dir_path, "pos")
## Negative comments tokenization ##
neg_dir_path = fr'C:\Users\YuvalZiv\Desktop\nlp project\train\neg'
neg_tokens_sentences, comments_type_list_neg = make_clean_comments(neg_dir_path, "neg")
    

In [3]:
#preparing the labled data
data_pos = {'comment':pos_tokens_sentences,
        'lable':comments_type_list_pos}
data_neg = {'comment':neg_tokens_sentences,
        'lable':comments_type_list_neg}

#creating a dataset of all the data together
dataset = pd.DataFrame(data_pos)
df_temp = pd.DataFrame(data_neg)
dataset.append(df_temp)


,comment,lable
0,"[bromwell, high, cartoon, comedy, ran, time, p...",1
1,"[homelessness, houselessness, george, carlin, ...",1
2,"[brilliant, overacting, lesley, ann, warren, b...",1
3,"[easily, underrated, film, inn, brooks, cannon...",1
4,"[typical, mel, brooks, film, much, less, slaps...",1
...,...,...
18745,"[towards, end, movie, felt, technical, felt, l...",0
18746,"[kind, movie, enemies, content, watch, time, b...",0
18747,"[saw, descent, last, night, stockholm, film, f...",0
18748,"[films, pick, pound, turn, rather, good, rd, c...",0


In [4]:
#creating a train set -> I tried also with random and stuff and it didn't work :(
k_folds=5
kfold = KFold(n_splits=k_folds, shuffle=True)
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=test_subsampler)

In [6]:
train_subsampler